# Face Detection using MTCNN

We try to use another face detection method to see where we get optimal performance. This time we use [MTCNN](https://github.com/ipazc/mtcnn). Turns out, this works better but takes much longer than OpenCV default method.

In [2]:
import numpy as np
import glob
import json
import cv2
from mtcnn import MTCNN

Using TensorFlow backend.
/Users/aniket/anaconda3/envs/aniket1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/aniket/anaconda3/envs/aniket1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/aniket/anaconda3/envs/aniket1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/aniket/

The initial process is same as before. So we just copy from the other code.

In [5]:
ana_dir_ofc = '/data/anaconda3/envs/aniket1/share/opencv4/haarcascades/'
ana_dir_mac = '/Users/aniket/anaconda3/envs/aniket1/share/OpenCV/haarcascades/'
basedir_ofc = '/data/Kaggle/DeepFake/'
basedir_mac = '/Users/aniket/KaggleData/DeepFake/'

basedir = basedir_mac
ana_dir = ana_dir_mac
#get all video filenames
files = glob.glob(basedir+'sample_data/train_sample_videos/*.mp4')
#read json file 
json_file = basedir+'sample_data/train_sample_videos/metadata.json'

with open(json_file, 'r') as f:
    json_dict = json.load(f)

nvids = len(files)     
vid_fname = ""
detector = MTCNN()

Now the loop over videos. This part is going to be different.

In [13]:
#loop over videos - capture image info as vectors
for i in np.arange(10, 11) :     
    #start capturing images 
    print('Capturing frames from : {}'.format(files[i]))
    vid_fname = files[i].rsplit('/')[-1]
    print(json_dict[vid_fname]['label'])
    cap = cv2.VideoCapture(files[i]) #basedir+'sample_data/train_sample_videos/bmehkyanbj.mp4'
    print('frame size = {}, {}, fps = {}'.format(cap.get(3), cap.get(4), cap.get(5)))
    c = 0
    gray0 = None
    while(cap.isOpened()):
        ret, frame = cap.read()
        if frame is not None : 
            c = c+1
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            ffaces = detector.detect_faces(rgb)
            if ffaces is not None :
                print(ffaces[0]['box'])
                x, y, w, h = ffaces[0]['box']
                xl, xr, yl, yr = x, x+w, y, y+h #[int(0.95*x), int(1.05*(x+w)), int(0.8*y), int(1.1*(y+h))]
                gray = cv2.rectangle(gray, (xl, yl), (xr, yr), (255, 0, 0), 2)
#             for (x,y,w,h) in np.asarray(ffaces[0]['box']):
#                 print('x = {}, y = {}, w = {}, h = {}'.format(x, y, w, h))
#                 face_arr.append(gray[xl:xr, yl:yr])
            if c > 1 : 
                cv2.imshow('frame', gray)      # - gray0 + 255
            else : 
                cv2.imshow('frame', gray)
#             gray0 = gray.copy()
            if cv2.waitKey(0) & 0xFF == ord('q'):
                break
        else : 
            break
        
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    print('num frames = {}'.format(c))

Capturing frames from : /Users/aniket/KaggleData/DeepFake/sample_data/train_sample_videos/atyntldecu.mp4
FAKE
frame size = 1920.0, 1080.0, fps = 29.97
[1028, 169, 235, 298]
[1019, 166, 239, 303]
[1015, 167, 230, 294]
[996, 158, 242, 304]
[987, 159, 237, 304]
[982, 171, 228, 287]
num frames = 6
